<a href="https://colab.research.google.com/github/RiddhimanRaut/Deep-Learning-based-CPR-estimation/blob/main/CPRAVG_SARIMAX_DAILY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pmdarima

     |████████████████████████████████| 1.5 MB 7.8 MB/s 
     |████████████████████████████████| 9.5 MB 61.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import pmdarima as pm

# Load specific forecasting tools
from statsmodels.tsa.statespace.sarimax import SARIMAX

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots
from pmdarima import auto_arima                              # for determining ARIMA orders

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")


In [17]:
path = "https://github.com/RiddhimanRaut/Deep-Learning-based-CPR-estimation/blob/main/excel_dataset.xlsx?raw=true"
df = pd.read_excel(path)
df = df.dropna()
date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')
df.index = date_time
df.head()

,CPRAVG,PROP,BUT,ETHYLENE,AI12201M,FLW,COT,TEMP,PRESS,O2,DRAFT,ETH
Date Time,,,,,,,,,,,,
2009-01-01 00:10:00,0.428151,18.960609,3.973630,29.732470,0.014431,30711.65039,820.118210,632.399261,0.466357,1.444160,-5.616851,75.809700
2009-01-01 00:20:00,0.427783,18.960360,3.973562,30.123590,0.014822,30700.66016,819.662811,633.057602,0.475288,1.413894,-5.540349,75.810066
2009-01-01 00:30:00,0.427432,19.009649,4.181924,30.477831,0.012689,30689.66016,819.886444,631.841385,0.464559,1.515123,-5.628389,75.552841
2009-01-01 00:40:00,0.427082,19.155710,3.950392,30.425819,0.012580,30678.66016,819.660172,632.495224,0.471809,1.404055,-5.576591,75.580223
2009-01-01 00:50:00,0.426732,19.334640,3.689848,30.546061,0.012472,30667.66016,819.646805,632.388214,0.470504,1.191485,-5.274679,75.638023


In [23]:
number_of_days = (df.shape[0]//144)+1 #// for integer division
number_of_extra_rows = df.shape[0]%144
df_days = df.rolling(144).mean().dropna()
df_days = df_days.iloc[::144,:]
df_extra = df[86*144:df.shape[0]].mean()
df_days = df_days.append(df_extra,ignore_index=True)
df_days

,CPRAVG,PROP,BUT,ETHYLENE,AI12201M,FLW,COT,TEMP,PRESS,O2,DRAFT,ETH
0,0.437426,18.989021,4.212129,32.006993,0.010173,30654.824301,830.488018,640.112469,0.469527,0.794234,-4.467693,75.694686
1,0.448062,18.809630,4.524849,33.836544,0.010565,30625.732653,844.455551,651.105717,0.469758,0.469703,-4.170596,75.561543
2,0.450353,19.083289,4.726570,34.668045,0.012945,30705.664307,852.964052,657.452192,0.470218,0.333544,-4.249914,75.352648
3,0.451834,18.531180,4.659772,34.896273,0.012922,30713.556058,854.393941,659.299923,0.469249,0.274911,-4.314996,75.919100
4,0.452508,17.260066,4.367844,34.849715,0.013748,30754.496406,854.443389,659.779801,0.469077,0.319588,-4.264858,77.543971
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0.516196,22.787623,4.144163,32.998388,0.012000,28634.796591,853.104591,651.138966,0.487782,0.976843,-4.962001,71.868771
83,0.516829,24.152432,4.141898,31.950313,0.012233,28754.501818,853.112442,650.101372,0.488156,1.030537,-4.959895,70.069474
84,0.517241,23.390510,4.132008,32.028396,0.012171,28687.527385,853.108998,651.155280,0.488213,1.037697,-4.961618,71.159993
85,0.517645,23.003283,4.141829,31.995858,0.012024,28685.423422,853.136709,651.738798,0.488620,0.917159,-4.959742,71.677207
